In [1]:
from functions import g_D_symbolic_coefficients_dict
from functions import restore_matrices
from functions import Augmented_Lagrangian
from functions import update_Lagrangian_coefficients
from functions import jac

In [288]:
D = 4
L = 6
d = 4
rho = 10

In [289]:
expanded_result, coefficients_dict,polynomial,x = g_D_symbolic_coefficients_dict(D)
print("Expanded Result:")
print(expanded_result)
print("\nCoefficients Dictionary:")
for vars_tuple, coeff in coefficients_dict.items():
    print(f"Coefficient of {vars_tuple}: {coeff}")

orders_list = list(coefficients_dict.keys())
coefficients_list = list(coefficients_dict.values())
print(orders_list)
print(coefficients_list)

Expanded Result:
2.0*x1**4 + x1**3/64 + 3*x1**2*x2/64 + 3*x1**2*x3/64 + 3*x1**2*x4/64 - 2.0*x1**2 + 3*x1*x2**2/64 + 3*x1*x2*x3/32 + 3*x1*x2*x4/32 + 3*x1*x3**2/64 + 3*x1*x3*x4/32 + 3*x1*x4**2/64 + 2.0*x2**4 + x2**3/64 + 3*x2**2*x3/64 + 3*x2**2*x4/64 - 2.0*x2**2 + 3*x2*x3**2/64 + 3*x2*x3*x4/32 + 3*x2*x4**2/64 + 2.0*x3**4 + x3**3/64 + 3*x3**2*x4/64 - 2.0*x3**2 + 3*x3*x4**2/64 + 2.0*x4**4 + x4**3/64 - 2.0*x4**2 + 1.0

Coefficients Dictionary:
Coefficient of (4, 0, 0, 0): 2.00000000000000
Coefficient of (3, 0, 0, 0): 1/64
Coefficient of (2, 1, 0, 0): 3/64
Coefficient of (2, 0, 1, 0): 3/64
Coefficient of (2, 0, 0, 1): 3/64
Coefficient of (2, 0, 0, 0): -2.00000000000000
Coefficient of (1, 2, 0, 0): 3/64
Coefficient of (1, 1, 1, 0): 3/32
Coefficient of (1, 1, 0, 1): 3/32
Coefficient of (1, 0, 2, 0): 3/64
Coefficient of (1, 0, 1, 1): 3/32
Coefficient of (1, 0, 0, 2): 3/64
Coefficient of (0, 4, 0, 0): 2.00000000000000
Coefficient of (0, 3, 0, 0): 1/64
Coefficient of (0, 2, 1, 0): 3/64
Coefficien

In [292]:
from scipy.integrate import quad
# This block is for initialization according to the article, µ, {R_i^l, S_i^l},is initialized randomly using a distribution U ([−1, 1]) uniform over [−1, 1].

# def function of the moments in M_d
def f(x,i,j):
    return 1/2*x**(i+j)

# def function of the function in M_{d-1}
def g(x,i,j):
    return 1/2*(1-x**2)*x**(i+j)

#function generate M_d(mu_{i}^{l})
def M_d(d):
    matrix = np.zeros((d+1, d+1))
    for i in range(d+1):
        for j in range(d+1):
            matrix[i,j],_= quad(f,-1,1,args=(i,j))
    return matrix

# function generate M_d(mu_{i}^{l})
def M_d_1(d):
    matrix = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            matrix[i,j],_= quad(g,-1,1,args=(i,j))
    return matrix

In [310]:
import numpy as np
x_M_D_L_list = [[] for _ in range(D)]
x_M_D_1_L_list = [[] for _ in range(D)]
x_R_L_list = [[] for _ in range(D)]
x_S_L_list = [[] for _ in range(D)]
for l in range(L):
    for i in range(D):
        x_M_D_L_list[i].append(M_d(d))
        # x_M_D_1_L_list[i].append(M_d_1(d))
        # x_M_D_L_list[i].append(np.random.uniform(-1, 1, size=(d+1, d+1)))
        # x_M_D_1_L_list[i].append(np.random.uniform(-1, 1, size=(d, d)))
        x_R_L_list[i].append(np.random.uniform(-1, 1, size=(d+1, d+1)))
        # x_S_L_list[i].append(np.random.uniform(-1, 1, size=(d, d)))

x_matrix_list_new = x_M_D_L_list+x_R_L_list+x_M_D_1_L_list+x_S_L_list
x_input = np.array([])
for matrix_index in range(len(x_matrix_list_new)):
    for l in range(L):
        flattened_array = x_matrix_list_new[matrix_index][l].flatten()
        x_input = np.concatenate((x_input, flattened_array))
print(len(x_input))
print(x_input)
Lagrangian_coefficient = 0
rho = 10

1968
[ 1.          0.          0.33333333 ... -0.51895337 -0.85921051
  0.58391704]


In [317]:
from functools import partial
aug_lagrangian_partial = partial(Augmented_Lagrangian, d=d, D=D, L=L, orders_list=orders_list,
                                 coefficients_list=coefficients_list,
                                 Lagrangian_coefficient=Lagrangian_coefficient, rho=rho)
jac_partial = partial(jac, d=d, D=D, L=L, orders_list=orders_list,
                      coefficients_list=coefficients_list,
                      Lagrangian_coefficient=Lagrangian_coefficient, rho=rho)
print(aug_lagrangian_partial(x_input))

1012.13531623709


In [318]:
print(aug_lagrangian_partial.keywords['Lagrangian_coefficient'])
print(aug_lagrangian_partial.keywords['rho'])

-43.33819055719816
10


In [319]:
print(jac_partial(x_input))

[-45.0420025012992, -43.2801781538308, -44.2630926599667, -43.3542377548604, -42.2593899132725, -43.2801781538308, -44.7361544599697, -43.3381120162397, -42.2268379392144, -43.36584519384944, -44.2630926599667, -43.3381120162397, -42.8336112858769, -43.31865416629755, -43.07011762423956, -43.3542377548604, -42.2268379392144, -43.31865416629755, -43.41970020695173, -43.25612270994627, -42.2593899132725, -43.36584519384944, -43.07011762423956, -43.25612270994627, -43.403183461861246, -46.0463389383274, -43.1736760373218, -44.4488286574876, -43.2919213574966, -41.9744856513809, -43.1736760373218, -44.6448641618287, -43.4090545017859, -42.2101304687332, -43.28219370706522, -44.4488286574876, -43.4090545017859, -42.6764217771362, -43.40124176605188, -43.02358180447065, -43.2919213574966, -42.2101304687332, -43.40124176605188, -43.31989595456564, -43.32483270926916, -41.9744856513809, -43.28219370706522, -43.02358180447065, -43.32483270926916, -43.491362275318735, -45.3525217520871, -43.3238

In [320]:
print(x_M_D_L_list)

[[array([[ 0.99242678,  0.09536652,  0.58437896,  0.07654862, -0.06913904],
       [ 0.09536652,  0.59392773, -0.01078909, -0.10138644, -0.02510258],
       [ 0.58437896, -0.01078909,  0.27450562,  0.02698524,  0.07210682],
       [ 0.07654862, -0.10138644,  0.02698524,  0.32137211, -0.16051595],
       [-0.06913904, -0.02510258,  0.07210682, -0.16051595,  0.26265344]]), array([[ 9.62946801e-01, -2.93820152e-02,  6.31042288e-01,
        -7.16224310e-04, -1.11652664e-01],
       [-2.93820152e-02,  5.53800508e-01, -1.46229303e-01,
        -9.01960827e-02,  1.19827732e-01],
       [ 6.31042288e-01, -1.46229303e-01,  2.60150504e-01,
         8.07546735e-02,  4.26297554e-02],
       [-7.16224310e-04, -9.01960827e-02,  8.07546735e-02,
         2.39345182e-01,  1.49624919e-01],
       [-1.11652664e-01,  1.19827732e-01,  4.26297554e-02,
         1.49624919e-01,  2.28354465e-01]]), array([[ 0.98076133, -0.07427866,  0.51084002, -0.17827736, -0.13529934],
       [-0.07427866,  0.31872663, -0.063

In [309]:
def generate_bounds(x):
    x_len = len(x)
    bounds_list = []
    for i in range(x_len):
        if i<L*D*(2*d+2)**2:
            bounds_list.append([-1,1])
        else:
            bounds_list.append([None,None])
    return(bounds_list)
bounds = generate_bounds(x_input)

In [321]:
from scipy.optimize import minimize


result = minimize(aug_lagrangian_partial, x0=x_input,
                  method='L-BFGS-B',
                  jac=jac_partial,
                  options={
                      'gtol': 1e-2,             # Stopping criterion (relative gradient)
                      'ftol': 1e-5,             # Stopping criterion (absolute value)
                  })

print("-"*40)
print("Minimum value of the Augmented Lagrangian function:", result.fun)
print("Was the optimization successful?", result.success)
print("Number of iterations:", result.nit)
print(result.message)
Lagrangian_coefficient = update_Lagrangian_coefficients(d,D,L,result.x,Lagrangian_coefficient,rho)
print("-"*40)
# rho = abs(Lagrangian_coefficient)*5
x_input = result.x
print("Lagrangian_coefficient:",Lagrangian_coefficient)

print("-"*40)
x_M_D_L_list,_,_,_= restore_matrices(x_input,d,D,L)

l_product_list = []
for l in range(L):
    moment_product = 1
    for i in range(D):
        moment_product *= x_M_D_L_list[i][l][0,0]
    l_product_list.append(moment_product)

max_index = l_product_list.index(max(l_product_list))

Denominator_product = 1
for i in range(D):
    Denominator_product *=  x_M_D_L_list[i][max_index][0,0]

x_output = {x[i]: x_M_D_L_list[i][max_index][0,1]/Denominator_product  for i in range(D)}
polynomial_values = polynomial.subs(x_output)

print("The final point: ",tuple(x_output.values()))
print("Objective function value:", polynomial_values)

----------------------------------------
Minimum value of the Augmented Lagrangian function: -37376.7461586485
Was the optimization successful? True
Number of iterations: 13
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
----------------------------------------
Lagrangian_coefficient: 11402.870135635516
----------------------------------------
The final point:  (1.0, 1.0, 1.0, 1.0)
Objective function value: 2.00000000000000
